In [ ]:
Import library

    import numpy as np
    import matplotlib.pyplot as plt
    import pandas as pd
    from sklearn.preprocessing import MinMaxScaler
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import LSTM
    from tensorflow.keras.layers import Dense
    from tensorflow.keras.layers import Dropout

    2023-05-22 18:05:35.874947: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
    To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
    2023-05-22 18:05:36.470463: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
    2023-05-22 18:05:36.470515: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
    2023-05-22 18:05:36.470521: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.

3. Data Collection

3.0 import the data

    dataset_train = pd.read_csv('Google_Stock_Price_Train.csv')

    dataset_train.head()

             Date    Open    High     Low   Close      Volume
    0  01/03/2012  325.25  332.83  324.97  663.59   7,380,500
    1  01/04/2012  331.27  333.87  329.08  666.45   5,749,400
    2  01/05/2012  329.83  330.75  326.89  657.21   6,590,300
    3  01/06/2012  328.34  328.77  323.68  648.24   5,405,900
    4  01/09/2012  322.04  322.29  309.46  620.76  11,688,800

    #keras only takes numpy array
    training_set = dataset_train.iloc[:, 1: 2].values

    training_set.shape

    (1509, 1)

3.1 Feature scaling

    sc = MinMaxScaler(feature_range = (0, 1))
    #fit: get min/max of train data
    training_set_scaled = sc.fit_transform(training_set)

3.2 Data structure creation

-   taking the reference of past 60 days of data to predict the future
    stock price.
-   It is observed that taking 60 days of past data gives us best
    results.
-   In this data set 60 days of data means 3 months of data.
-   Every month as 20 days of Stock price.
-   X train will have data of 60 days prior to our date and y train will
    have data of one day after our date

    ## 60 timesteps and 1 output
    X_train = []
    y_train = []
    for i in range(60, len(training_set_scaled)):
        X_train.append(training_set_scaled[i-60: i, 0])
        y_train.append(training_set_scaled[i, 0])

    X_train, y_train = np.array(X_train), np.array(y_train)

    X_train.shape

    (1449, 60)

    y_train.shape

    (1449,)

3.3 Data reshaping

    X_train = np.reshape(X_train, newshape =
                         (X_train.shape[0], X_train.shape[1], 1))

1.  Number of stock prices - 1449
2.  Number of time steps - 60
3.  Number of Indicator - 1

    X_train.shape

    (1449, 60, 1)

4. Exploratory Data Analysis

    plt.figure(figsize=(18, 8))
    plt.plot(dataset_train['Open'])
    plt.title("Google Stock Open Prices")
    plt.xlabel("Time (oldest -> latest)")
    plt.ylabel("Stock Open Price")
    plt.show()

[]

    plt.figure(figsize=(18, 8))
    plt.plot(dataset_train['Low'])
    plt.title("Google Stock Low Prices")
    plt.xlabel("Time (oldest -> latest)")
    plt.ylabel("Stock Lowest Price")
    plt.show()

[]

5. Create & Fit Model

5.1 Create model

    regressor = Sequential()
    #add 1st lstm layer
    regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
    regressor.add(Dropout(rate = 0.2))

    ##add 2nd lstm layer: 50 neurons
    regressor.add(LSTM(units = 50, return_sequences = True))
    regressor.add(Dropout(rate = 0.2))

    ##add 3rd lstm layer
    regressor.add(LSTM(units = 50, return_sequences = True))
    regressor.add(Dropout(rate = 0.2))

    ##add 4th lstm layer
    regressor.add(LSTM(units = 50, return_sequences = False))
    regressor.add(Dropout(rate = 0.2))

    ##add output layer
    regressor.add(Dense(units = 1))

    2023-05-22 18:05:38.589877: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
    2023-05-22 18:05:38.917051: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
    2023-05-22 18:05:38.917079: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1934] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
    Skipping registering GPU devices...
    2023-05-22 18:05:38.917439: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
    To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.

    regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

5.2 Model fit

    regressor.fit(x = X_train, y = y_train, batch_size = 32, epochs = 100)

    Epoch 1/100
    46/46 [==============================] - 8s 63ms/step - loss: 0.0218
    Epoch 2/100
    46/46 [==============================] - 3s 63ms/step - loss: 0.0042
    Epoch 3/100
    46/46 [==============================] - 3s 68ms/step - loss: 0.0041
    Epoch 4/100
    46/46 [==============================] - 3s 69ms/step - loss: 0.0034
    Epoch 5/100
    46/46 [==============================] - 3s 71ms/step - loss: 0.0031
    Epoch 6/100
    46/46 [==============================] - 3s 64ms/step - loss: 0.0030
    Epoch 7/100
    46/46 [==============================] - 3s 71ms/step - loss: 0.0034
    Epoch 8/100
    46/46 [==============================] - 3s 70ms/step - loss: 0.0032
    Epoch 9/100
    46/46 [==============================] - 3s 67ms/step - loss: 0.0029
    Epoch 10/100
    46/46 [==============================] - 3s 59ms/step - loss: 0.0038
    Epoch 11/100
    46/46 [==============================] - 3s 59ms/step - loss: 0.0027
    Epoch 12/100
    46/46 [==============================] - 3s 63ms/step - loss: 0.0029
    Epoch 13/100
    46/46 [==============================] - 3s 61ms/step - loss: 0.0026
    Epoch 14/100
    46/46 [==============================] - 3s 71ms/step - loss: 0.0024
    Epoch 15/100
    46/46 [==============================] - 3s 68ms/step - loss: 0.0024
    Epoch 16/100
    46/46 [==============================] - 3s 68ms/step - loss: 0.0024
    Epoch 17/100
    46/46 [==============================] - 3s 66ms/step - loss: 0.0024
    Epoch 18/100
    46/46 [==============================] - 3s 66ms/step - loss: 0.0024
    Epoch 19/100
    46/46 [==============================] - 3s 64ms/step - loss: 0.0024
    Epoch 20/100
    46/46 [==============================] - 3s 68ms/step - loss: 0.0024
    Epoch 21/100
    46/46 [==============================] - 3s 68ms/step - loss: 0.0020
    Epoch 22/100
    46/46 [==============================] - 3s 72ms/step - loss: 0.0021
    Epoch 23/100
    46/46 [==============================] - 3s 69ms/step - loss: 0.0020
    Epoch 24/100
    46/46 [==============================] - 3s 70ms/step - loss: 0.0024
    Epoch 25/100
    46/46 [==============================] - 3s 75ms/step - loss: 0.0021
    Epoch 26/100
    46/46 [==============================] - 3s 73ms/step - loss: 0.0019
    Epoch 27/100
    46/46 [==============================] - 3s 71ms/step - loss: 0.0018
    Epoch 28/100
    46/46 [==============================] - 3s 75ms/step - loss: 0.0018
    Epoch 29/100
    46/46 [==============================] - 3s 73ms/step - loss: 0.0018
    Epoch 30/100
    46/46 [==============================] - 3s 70ms/step - loss: 0.0021
    Epoch 31/100
    46/46 [==============================] - 3s 71ms/step - loss: 0.0017
    Epoch 32/100
    46/46 [==============================] - 3s 66ms/step - loss: 0.0016
    Epoch 33/100
    46/46 [==============================] - 3s 73ms/step - loss: 0.0018
    Epoch 34/100
    46/46 [==============================] - 3s 74ms/step - loss: 0.0018
    Epoch 35/100
    46/46 [==============================] - 3s 74ms/step - loss: 0.0015
    Epoch 36/100
    46/46 [==============================] - 3s 70ms/step - loss: 0.0021
    Epoch 37/100
    46/46 [==============================] - 3s 64ms/step - loss: 0.0019
    Epoch 38/100
    46/46 [==============================] - 3s 67ms/step - loss: 0.0015
    Epoch 39/100
    46/46 [==============================] - 3s 68ms/step - loss: 0.0016
    Epoch 40/100
    46/46 [==============================] - 3s 66ms/step - loss: 0.0015
    Epoch 41/100
    46/46 [==============================] - 3s 65ms/step - loss: 0.0016
    Epoch 42/100
    46/46 [==============================] - 3s 66ms/step - loss: 0.0016
    Epoch 43/100
    46/46 [==============================] - 3s 62ms/step - loss: 0.0013
    Epoch 44/100
    46/46 [==============================] - 3s 64ms/step - loss: 0.0015
    Epoch 45/100
    46/46 [==============================] - 3s 63ms/step - loss: 0.0014
    Epoch 46/100
    46/46 [==============================] - 3s 64ms/step - loss: 0.0017
    Epoch 47/100
    46/46 [==============================] - 3s 70ms/step - loss: 0.0016
    Epoch 48/100
    46/46 [==============================] - 3s 72ms/step - loss: 0.0014
    Epoch 49/100
    46/46 [==============================] - 3s 69ms/step - loss: 0.0013
    Epoch 50/100
    46/46 [==============================] - 3s 62ms/step - loss: 0.0014
    Epoch 51/100
    46/46 [==============================] - 3s 67ms/step - loss: 0.0015
    Epoch 52/100
    46/46 [==============================] - 3s 66ms/step - loss: 0.0014
    Epoch 53/100
    46/46 [==============================] - 3s 63ms/step - loss: 0.0013
    Epoch 54/100
    46/46 [==============================] - 3s 61ms/step - loss: 0.0014
    Epoch 55/100
    46/46 [==============================] - 3s 65ms/step - loss: 0.0012
    Epoch 56/100
    46/46 [==============================] - 3s 61ms/step - loss: 0.0012
    Epoch 57/100
    46/46 [==============================] - 3s 65ms/step - loss: 0.0013
    Epoch 58/100
    46/46 [==============================] - 3s 63ms/step - loss: 0.0012
    Epoch 59/100
    46/46 [==============================] - 3s 61ms/step - loss: 0.0012
    Epoch 60/100
    46/46 [==============================] - 3s 63ms/step - loss: 0.0012
    Epoch 61/100
    46/46 [==============================] - 3s 60ms/step - loss: 0.0011
    Epoch 62/100
    46/46 [==============================] - 3s 60ms/step - loss: 0.0011
    Epoch 63/100
    46/46 [==============================] - 3s 62ms/step - loss: 0.0012
    Epoch 64/100
    46/46 [==============================] - 3s 65ms/step - loss: 0.0014
    Epoch 65/100
    46/46 [==============================] - 3s 67ms/step - loss: 0.0012
    Epoch 66/100
    46/46 [==============================] - 3s 63ms/step - loss: 0.0013
    Epoch 67/100
    46/46 [==============================] - 3s 64ms/step - loss: 0.0011
    Epoch 68/100
    46/46 [==============================] - 3s 60ms/step - loss: 0.0011
    Epoch 69/100
    46/46 [==============================] - 3s 70ms/step - loss: 0.0012
    Epoch 70/100
    46/46 [==============================] - 3s 66ms/step - loss: 0.0012
    Epoch 71/100
    46/46 [==============================] - 3s 64ms/step - loss: 0.0011
    Epoch 72/100
    46/46 [==============================] - 3s 67ms/step - loss: 0.0010
    Epoch 73/100
    46/46 [==============================] - 3s 62ms/step - loss: 0.0011
    Epoch 74/100
    46/46 [==============================] - 3s 65ms/step - loss: 0.0011
    Epoch 75/100
    46/46 [==============================] - 3s 65ms/step - loss: 0.0010
    Epoch 76/100
    46/46 [==============================] - 3s 66ms/step - loss: 9.8192e-04
    Epoch 77/100
    46/46 [==============================] - 3s 62ms/step - loss: 0.0011
    Epoch 78/100
    46/46 [==============================] - 3s 67ms/step - loss: 0.0011
    Epoch 79/100
    46/46 [==============================] - 3s 68ms/step - loss: 0.0011
    Epoch 80/100
    46/46 [==============================] - 3s 67ms/step - loss: 0.0010
    Epoch 81/100
    46/46 [==============================] - 3s 67ms/step - loss: 0.0010
    Epoch 82/100
    46/46 [==============================] - 3s 65ms/step - loss: 0.0010
    Epoch 83/100
    46/46 [==============================] - 3s 69ms/step - loss: 9.7680e-04
    Epoch 84/100
    46/46 [==============================] - 3s 69ms/step - loss: 0.0011
    Epoch 85/100
    46/46 [==============================] - 3s 67ms/step - loss: 0.0010
    Epoch 86/100
    46/46 [==============================] - 3s 67ms/step - loss: 0.0011
    Epoch 87/100
    46/46 [==============================] - 3s 67ms/step - loss: 9.6028e-04
    Epoch 88/100
    46/46 [==============================] - 3s 67ms/step - loss: 9.9195e-04
    Epoch 89/100
    46/46 [==============================] - 3s 66ms/step - loss: 0.0012
    Epoch 90/100
    46/46 [==============================] - 3s 68ms/step - loss: 0.0010
    Epoch 91/100
    46/46 [==============================] - 3s 63ms/step - loss: 0.0011
    Epoch 92/100
    46/46 [==============================] - 3s 67ms/step - loss: 0.0010
    Epoch 93/100
    46/46 [==============================] - 3s 62ms/step - loss: 9.7312e-04
    Epoch 94/100
    46/46 [==============================] - 3s 65ms/step - loss: 8.6645e-04
    Epoch 95/100
    46/46 [==============================] - 3s 67ms/step - loss: 9.2218e-04
    Epoch 96/100
    46/46 [==============================] - 3s 68ms/step - loss: 0.0010
    Epoch 97/100
    46/46 [==============================] - 3s 70ms/step - loss: 0.0012
    Epoch 98/100
    46/46 [==============================] - 3s 65ms/step - loss: 9.1810e-04
    Epoch 99/100
    46/46 [==============================] - 3s 68ms/step - loss: 8.5810e-04
    Epoch 100/100

    46/46 [==============================] - 3s 65ms/step - loss: 9.4933e-04

    <keras.callbacks.History at 0x7f1c7f8ca1d0>

6. Model evaluation

6.1 Read and convert

    dataset_test = pd.read_csv('Google_Stock_Price_Test.csv')

    dataset_test.head()

             Date         Open         High          Low        Close   Volume
    0  02/01/2018  1048.339966  1066.939941  1045.229980  1065.000000  1237600
    1  03/01/2018  1064.310059  1086.290039  1063.209961  1082.479980  1430200
    2  04/01/2018  1088.000000  1093.569946  1084.001953  1086.400024  1004600
    3  05/01/2018  1094.000000  1104.250000  1092.000000  1102.229980  1279100
    4  08/01/2018  1102.229980  1111.270020  1101.619995  1106.939941  1047600

    #keras only takes numpy array
    real_stock_price = dataset_test.iloc[:, 1: 2].values
    real_stock_price.shape

    (125, 1)

6.2 Concat and convert

    #vertical concat use 0, horizontal uses 1
    dataset_total = pd.concat((dataset_train['Open'], dataset_test['Open']),
                              axis = 0)
    ##use .values to make numpy array
    inputs = dataset_total[len(dataset_total) - len(dataset_test) - 60:].values

6.3 Reshape and scale

    #reshape data to only have 1 col
    inputs = inputs.reshape(-1, 1)

    #scale input
    inputs = sc.transform(inputs)

    len(inputs)

    185

6.4 Create test data strucutre

    X_test = []
    for i in range(60, len(inputs)):
        X_test.append(inputs[i-60:i, 0])
    X_test = np.array(X_test)
    #add dimension of indicator
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

    X_test.shape

    (125, 60, 1)


    predicted_stock_price = regressor.predict(X_test)

    4/4 [==============================] - 1s 26ms/step

    #inverse the scaled value
    predicted_stock_price = sc.inverse_transform(predicted_stock_price)

    ##visualize the prediction and real price
    plt.plot(real_stock_price, color = 'red', label = 'Real price')
    plt.plot(predicted_stock_price, color = 'blue', label = 'Predicted price')

    plt.title('Google price prediction')
    plt.xlabel('Time')
    plt.ylabel('Price')
    plt.legend()
    plt.show()

[]
